# Taller 2 - Estadística II (R)

Este notebook implementa en R las 11 pruebas de hipótesis del Taller 2 usando el archivo `Combinado.csv`. 


#   Datos a usar
cuántitativas: 
1. P3094S3 : Cuánto ahorro por cultivar
2. P3087S1: Valor mensual por practicas o pasantías
3. P3095S3: Valor ahorra por criar animales
# Cualitativas
4. P3101: ¿Fue a reuniones familiares durante las ultimas 4 semanas? si o no

# Pruebas de Hipótesis a Desarrollar
1. Prueba de hipótesis para la media de una muestra con varianza conocida
2. Prueba de hipótesis para la proporción de una muestra
3. Prueba de hipótesis para la media de una muestra con varianza desconocida
4. Prueba de hipótesis para la diferencia de medias de dos muestras con varianzas conocidas
5. Prueba de hipótesis para la diferencia de medias de dos muestras con varianzas iguales y desconocidas
6. Prueba de hipótesis para la diferencia de medias de dos muestras con varianzas desiguales y desconocidas
7. Prueba de hipótesis para la diferencia de proporciones de dos muestras
8. Prueba de hipótesis para muestras dependientes
9. Prueba de bondad de ajuste
10. Prueba de independencia
11. Prueba de signos

### Librerias


In [100]:
# Setup: instalar y cargar librerías
required_pkgs <- c("tidyverse", "readr", "stringr", "broom", "ggplot2")
install_if_needed <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg, repos = "https://cloud.r-project.org")
  }
}
invisible(lapply(required_pkgs, install_if_needed))

suppressPackageStartupMessages({
  library(tidyverse)
  library(readr)
  library(stringr)
  library(broom)
  library(ggplot2)
})

cat("✅ Librerías listas:\n"); print(required_pkgs)


✅ Librerías listas:
[1] "tidyverse" "readr"     "stringr"   "broom"     "ggplot2"  


In [101]:
# Contenedor de resultados y export util
suppressPackageStartupMessages({
  library(jsonlite)
})
results <- list()


### Carga y limpieza de datos

In [102]:
# Carga y limpieza de datos
find_existing_path <- function(paths) {
  for (p in paths) {
    if (file.exists(p)) return(p)
  }
  stop("No se encontró 'Combinado.csv' en rutas esperadas")
}
paths <- c("Combinado.csv", "../Combinado.csv", "Taller 1/Combinado.csv", "../Taller 1/Combinado.csv")
csv_path <- find_existing_path(paths)
cat("✔ Archivo encontrado en:", csv_path, "\n")

df_raw <- readr::read_csv(
  csv_path,
  locale = locale(encoding = "UTF-8"),
  show_col_types = FALSE,
  col_types = cols(
    .default = col_guess(),
    P3094S3 = col_character(),
    P3087S1 = col_character(),
    P3095S3 = col_character(),
    P3101   = col_guess()
  )
)

clean_numeric <- function(x) {
  x_chr <- as.character(x)
  x_chr <- str_trim(x_chr)
  x_chr <- str_replace_all(x_chr, "\\s+", "")
  x_chr <- str_replace_all(x_chr, "[€$£'\"]", "")
  x_chr <- str_replace_all(x_chr, "\\.", "")
  x_chr <- str_replace_all(x_chr, ",", ".")
  suppressWarnings(as.numeric(x_chr))
}

# Codificación binaria de P3101: 1 = Sí, 2 = No
p3101_raw <- suppressWarnings(as.integer(df_raw$P3101))
p3101_bin <- case_when(
  p3101_raw == 1L ~ 1L,
  p3101_raw == 2L ~ 0L,
  TRUE ~ NA_integer_
)

df <- df_raw %>%
  mutate(
    P3094S3_clean = clean_numeric(P3094S3),
    P3087S1_clean = clean_numeric(P3087S1),
    P3095S3_clean = clean_numeric(P3095S3),
    P3101_bin     = p3101_bin
  )

cat("\nConteos tras limpieza (no NA):\n")
cat("P3087S1_clean:", sum(!is.na(df$P3087S1_clean)), "\n")
cat("P3094S3_clean:", sum(!is.na(df$P3094S3_clean)), "\n")
cat("P3095S3_clean:", sum(!is.na(df$P3095S3_clean)), "\n")
cat("\nDistribución P3101_bin (1=Sí,0=No):\n"); print(table(df$P3101_bin, useNA = "ifany"))


✔ Archivo encontrado en: ../Taller 1/Combinado.csv 

Conteos tras limpieza (no NA):
P3087S1_clean: 336 
P3094S3_clean: 2892 
P3095S3_clean: 4080 

Distribución P3101_bin (1=Sí,0=No):

     0      1 
163423   2918 


## Funciones para las 11 pruebas de hipótesis


In [103]:
z_test_media_conocida <- function(x, mu0, sigma, alternative = "two.sided", alpha = 0.05) {
  x <- na.omit(x); n <- length(x); if (n == 0) stop("Muestra vacía")
  x_bar <- mean(x); se <- sigma / sqrt(n); z_stat <- (x_bar - mu0) / se
  if (alternative == "two.sided") { p_value <- 2 * (1 - pnorm(abs(z_stat))); crit <- qnorm(1 - alpha/2) }
  else if (alternative == "greater") { p_value <- 1 - pnorm(z_stat); crit <- qnorm(1 - alpha) }
  else if (alternative == "less") { p_value <- pnorm(z_stat); crit <- qnorm(alpha) }
  else stop("alternative inválido")
  z_crit <- qnorm(1 - alpha/2); ci <- c(x_bar - z_crit * se, x_bar + z_crit * se)
  list(z = z_stat, p_value = p_value, crit = crit, mean = x_bar, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

z_test_proporcion <- function(x, p0, alternative = "two.sided", alpha = 0.05) {
  x <- na.omit(x); if (!all(x %in% c(0,1))) stop("x debe ser binaria 0/1")
  n <- length(x); p_hat <- mean(x); se <- sqrt(p0 * (1 - p0) / n); z_stat <- (p_hat - p0) / se
  if (alternative == "two.sided") { p_value <- 2 * (1 - pnorm(abs(z_stat))); crit <- qnorm(1 - alpha/2) }
  else if (alternative == "greater") { p_value <- 1 - pnorm(z_stat); crit <- qnorm(1 - alpha) }
  else if (alternative == "less") { p_value <- pnorm(z_stat); crit <- qnorm(alpha) }
  else stop("alternative inválido")
  z_crit <- qnorm(1 - alpha/2); se_ci <- sqrt(p_hat * (1 - p_hat) / n); ci <- c(p_hat - z_crit * se_ci, p_hat + z_crit * se_ci)
  list(z = z_stat, p_value = p_value, crit = crit, phat = p_hat, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

t_test_media_desconocida <- function(x, mu0, alternative = "two.sided", alpha = 0.05) {
  x <- na.omit(x); n <- length(x); if (n < 2) stop("Se requieren al menos 2 observaciones")
  x_bar <- mean(x); s <- sd(x); se <- s / sqrt(n); t_stat <- (x_bar - mu0) / se; df <- n - 1
  if (alternative == "two.sided") { p_value <- 2 * (1 - pt(abs(t_stat), df)); crit <- qt(1 - alpha/2, df) }
  else if (alternative == "greater") { p_value <- 1 - pt(t_stat, df); crit <- qt(1 - alpha, df) }
  else if (alternative == "less") { p_value <- pt(t_stat, df); crit <- qt(alpha, df) }
  else stop("alternative inválido")
  t_crit <- qt(1 - alpha/2, df); ci <- c(x_bar - t_crit * se, x_bar + t_crit * se)
  list(t = t_stat, p_value = p_value, crit = crit, df = df, mean = x_bar, s = s, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

z_test_dif_medias_conocidas <- function(x1, x2, sigma1, sigma2, mu0 = 0, alternative = "two.sided", alpha = 0.05) {
  x1 <- na.omit(x1); x2 <- na.omit(x2); n1 <- length(x1); n2 <- length(x2)
  x1_bar <- mean(x1); x2_bar <- mean(x2); se <- sqrt(sigma1^2/n1 + sigma2^2/n2); z_stat <- ((x1_bar - x2_bar) - mu0) / se
  if (alternative == "two.sided") { p_value <- 2 * (1 - pnorm(abs(z_stat))); crit <- qnorm(1 - alpha/2) }
  else if (alternative == "greater") { p_value <- 1 - pnorm(z_stat); crit <- qnorm(1 - alpha) }
  else if (alternative == "less") { p_value <- pnorm(z_stat); crit <- qnorm(alpha) }
  else stop("alternative inválido")
  z_crit <- qnorm(1 - alpha/2); ci <- c((x1_bar - x2_bar) - z_crit * se, (x1_bar - x2_bar) + z_crit * se)
  list(z = z_stat, p_value = p_value, crit = crit, mean1 = x1_bar, mean2 = x2_bar, diff = x1_bar - x2_bar, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

t_test_dif_medias_iguales <- function(x1, x2, mu0 = 0, alternative = "two.sided", alpha = 0.05) {
  x1 <- na.omit(x1); x2 <- na.omit(x2); n1 <- length(x1); n2 <- length(x2); s1 <- sd(x1); s2 <- sd(x2)
  sp2 <- (((n1 - 1) * s1^2) + ((n2 - 1) * s2^2)) / (n1 + n2 - 2)
  se <- sqrt(sp2 * (1/n1 + 1/n2)); t_stat <- ((mean(x1) - mean(x2)) - mu0) / se; df <- n1 + n2 - 2
  if (alternative == "two.sided") { p_value <- 2 * (1 - pt(abs(t_stat), df)); crit <- qt(1 - alpha/2, df) }
  else if (alternative == "greater") { p_value <- 1 - pt(t_stat, df); crit <- qt(1 - alpha, df) }
  else if (alternative == "less") { p_value <- pt(t_stat, df); crit <- qt(alpha, df) }
  else stop("alternative inválido")
  t_crit <- qt(1 - alpha/2, df); ci <- c((mean(x1) - mean(x2)) - t_crit * se, (mean(x1) - mean(x2)) + t_crit * se)
  list(t = t_stat, p_value = p_value, crit = crit, df = df, diff = mean(x1) - mean(x2), sp2 = sp2, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

t_test_dif_medias_desiguales <- function(x1, x2, mu0 = 0, alternative = "two.sided", alpha = 0.05) {
  x1 <- na.omit(x1); x2 <- na.omit(x2); n1 <- length(x1); n2 <- length(x2); s1 <- sd(x1); s2 <- sd(x2)
  se <- sqrt(s1^2/n1 + s2^2/n2); df <- (s1^2/n1 + s2^2/n2)^2 / ((s1^2/n1)^2/(n1-1) + (s2^2/n2)^2/(n2-1))
  t_stat <- ((mean(x1) - mean(x2)) - mu0) / se
  if (alternative == "two.sided") { p_value <- 2 * (1 - pt(abs(t_stat), df)); crit <- qt(1 - alpha/2, df) }
  else if (alternative == "greater") { p_value <- 1 - pt(t_stat, df); crit <- qt(1 - alpha, df) }
  else if (alternative == "less") { p_value <- pt(t_stat, df); crit <- qt(alpha, df) }
  else stop("alternative inválido")
  t_crit <- qt(1 - alpha/2, df); ci <- c((mean(x1) - mean(x2)) - t_crit * se, (mean(x1) - mean(x2)) + t_crit * se)
  list(t = t_stat, p_value = p_value, crit = crit, df = df, diff = mean(x1) - mean(x2), se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

z_test_dif_proporciones <- function(x1, n1, x2, n2, p0 = 0, alternative = "two.sided", alpha = 0.05) {
  p1_hat <- x1 / n1; p2_hat <- x2 / n2; p_comb <- (x1 + x2) / (n1 + n2)
  se <- sqrt(p_comb * (1 - p_comb) * (1/n1 + 1/n2)); z_stat <- ((p1_hat - p2_hat) - p0) / se
  if (alternative == "two.sided") { p_value <- 2 * (1 - pnorm(abs(z_stat))); crit <- qnorm(1 - alpha/2) }
  else if (alternative == "greater") { p_value <- 1 - pnorm(z_stat); crit <- qnorm(1 - alpha) }
  else if (alternative == "less") { p_value <- pnorm(z_stat); crit <- qnorm(alpha) }
  else stop("alternative inválido")
  z_crit <- qnorm(1 - alpha/2); se_ci <- sqrt(p1_hat*(1-p1_hat)/n1 + p2_hat*(1-p2_hat)/n2); ci <- c((p1_hat - p2_hat) - z_crit * se_ci, (p1_hat - p2_hat) + z_crit * se_ci)
  list(z = z_stat, p_value = p_value, crit = crit, p1 = p1_hat, p2 = p2_hat, diff = p1_hat - p2_hat, p_comb = p_comb, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

t_test_pareado <- function(x1, x2, mu0 = 0, alternative = "two.sided", alpha = 0.05) {
  if (length(x1) != length(x2)) stop("Longitudes distintas")
  d <- x1 - x2; n <- length(d); d_bar <- mean(d, na.rm = TRUE); s_d <- sd(d, na.rm = TRUE); se <- s_d / sqrt(n)
  t_stat <- (d_bar - mu0) / se; df <- n - 1
  if (alternative == "two.sided") { p_value <- 2 * (1 - pt(abs(t_stat), df)); crit <- qt(1 - alpha/2, df) }
  else if (alternative == "greater") { p_value <- 1 - pt(t_stat, df); crit <- qt(1 - alpha, df) }
  else if (alternative == "less") { p_value <- pt(t_stat, df); crit <- qt(alpha, df) }
  else stop("alternative inválido")
  t_crit <- qt(1 - alpha/2, df); ci <- c(d_bar - t_crit * se, d_bar + t_crit * se)
  list(t = t_stat, p_value = p_value, crit = crit, df = df, d_bar = d_bar, s_d = s_d, se = se, ci = ci, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

chi2_bondad_ajuste <- function(x, expected_props = NULL, alpha = 0.05) {
  x <- as.integer(na.omit(x)); if (length(x) == 0) stop("Muestra vacía")
  observed <- table(x); k <- length(observed); n <- sum(observed)
  if (is.null(expected_props)) expected_props <- rep(1/k, k) else {
    if (length(expected_props) != k) stop("Dimensiones no coinciden"); expected_props <- expected_props / sum(expected_props)
  }
  expected <- n * expected_props; chi2_stat <- sum((as.numeric(observed) - expected)^2 / expected)
  df <- k - 1; p_value <- 1 - pchisq(chi2_stat, df); crit <- qchisq(1 - alpha, df)
  list(chi2 = chi2_stat, p_value = p_value, crit = crit, df = df, observed = observed, expected = expected, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}

chi2_independencia <- function(x, y, alpha = 0.05) {
  tab <- table(x, y); test <- suppressWarnings(chisq.test(tab, correct = FALSE))
  list(chi2 = unname(test$statistic), p_value = unname(test$p.value), df = unname(test$parameter), expected = test$expected, observed = tab,
       crit = qchisq(1 - alpha, df = unname(test$parameter)), decision = ifelse(test$p.value < alpha, "Rechazar H0", "No rechazar H0"))
}

prueba_signos <- function(x, y, alternative = "two.sided", alpha = 0.05) {
  if (length(x) != length(y)) stop("Longitudes distintas")
  d <- x - y; pos <- sum(d > 0, na.rm = TRUE); neg <- sum(d < 0, na.rm = TRUE); emp <- sum(d == 0, na.rm = TRUE); n <- pos + neg
  if (n == 0) return(list(stat = 0, p_value = 1, pos = pos, neg = neg, emp = emp, n = n, decision = "No rechazar H0"))
  stat <- min(pos, neg)
  if (alternative == "two.sided") p_value <- 2 * pbinom(stat, n, 0.5) else if (alternative == "greater") p_value <- 1 - pbinom(pos - 1, n, 0.5) else if (alternative == "less") p_value <- pbinom(pos, n, 0.5) else stop("alternative inválido")
  list(stat = stat, p_value = p_value, pos = pos, neg = neg, emp = emp, n = n, decision = ifelse(p_value < alpha, "Rechazar H0", "No rechazar H0"))
}


## Aplicación de las pruebas


### Prueba 1: Media de una muestra con varianza conocida (Z)

- Hipótesis: H0: μ = μ0; H1: μ ≠ μ0 (o >, <)
- Variable: `P3095S3_clean` (ahorro por criar animales)
- Supuesto: σ conocida (usaremos 500000 como valor de referencia)
- Nivel de significancia: α = 0.05


In [104]:
mu0 <- 100000; sigma0 <- 500000
x <- na.omit(df$P3095S3_clean)
if (length(x) >= 1) {
  res_z1 <- z_test_media_conocida(x, mu0 = mu0, sigma = sigma0)
  print(res_z1)
  results$z_media_conocida <- res_z1
} else {
  cat("⚠️ No hay datos suficientes en P3095S3_clean para la prueba Z de media.\n")
}


$z
[1] 7.597453

$p_value
[1] 3.019807e-14

$crit
[1] 1.959964

$mean
[1] 159471.4

$se
[1] 7827.804

$ci
[1] 144129.2 174813.6

$decision
[1] "Rechazar H0"



In [105]:
# Inicializar contenedor de resultados
results <- list()


### Prueba 2: Proporción de una muestra (Z)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si la proporción poblacional de personas que asistieron a reuniones familiares (`P3101_bin`) difiere significativamente de p₀ = 0.5 (50%). Es adecuada porque tenemos una variable binaria (Sí/No) y queremos comparar la proporción observada con una hipótesis específica.

**Hipótesis:**
- H₀: p = 0.5 (el 50% de la población asistió a reuniones familiares)
- H₁: p ≠ 0.5 (la proporción poblacional difiere del 50%)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que la proporción real de asistencia a reuniones familiares difiere significativamente del 50%. Esto sugiere un patrón de asistencia no equilibrado en la población.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que la mitad de la población asiste a reuniones familiares.

**Supuestos clave:**
- La variable es binaria (0/1 o Sí/No)
- Las observaciones son independientes
- Muestra aleatoria de la población
- Tamaño muestral suficiente: np₀ ≥ 5 y n(1-p₀) ≥ 5
- Distribución muestral de la proporción aproximadamente normal

**Significado del estadístico y IC:**
- **Estadístico Z**: Mide cuántas desviaciones estándar se aleja la proporción muestral de p₀. Valores extremos indican evidencia contra H₀.
- **Intervalo de confianza**: Rango de valores plausibles para la proporción poblacional. Si p₀ no está en el IC, rechazamos H₀.


In [106]:
p0 <- 0.5
y <- na.omit(df$P3101_bin)
if (length(y) >= 1 && all(y %in% c(0L,1L))) {
  res_z2 <- z_test_proporcion(y, p0 = p0)
  print(res_z2)
  results$z_proporcion <- res_z2
} else {
  cat("⚠️ P3101_bin no disponible o no binaria.\n")
}


$z
[1] -393.54

$p_value
[1] 0

$crit
[1] 1.959964

$phat
[1] 0.01754228

$se
[1] 0.001225943

$ci
[1] 0.01691140 0.01817316

$decision
[1] "Rechazar H0"



### Prueba 3: Media de una muestra con varianza desconocida (t)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si la media poblacional del valor mensual por prácticas o pasantías (`P3087S1_clean`) difiere significativamente de μ₀ = 200,000 pesos. Es adecuada porque tenemos una variable cuantitativa continua pero no conocemos la varianza poblacional, por lo que usamos la distribución t.

**Hipótesis:**
- H₀: μ = 200,000 (la media poblacional del valor mensual por prácticas es 200,000 pesos)
- H₁: μ ≠ 200,000 (la media poblacional difiere de 200,000 pesos)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que el valor promedio mensual por prácticas en la población difiere significativamente de 200,000 pesos. Esto sugiere que las prácticas tienen un valor económico diferente al esperado.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que el valor promedio mensual es 200,000 pesos.

**Supuestos clave:**
- La variable sigue una distribución normal (o n ≥ 30 por el TCL)
- La desviación estándar poblacional es desconocida
- Las observaciones son independientes
- Muestra aleatoria de la población
- Se usa la varianza muestral para estimar σ

**Significado del estadístico y IC:**
- **Estadístico t**: Similar al Z pero usa la desviación estándar muestral. Se distribuye como t con n-1 grados de libertad.
- **Intervalo de confianza**: Rango de valores plausibles para la media poblacional usando la distribución t. Más amplio que el IC con Z debido a la incertidumbre adicional.


In [107]:
mu0 <- 200000
x <- na.omit(df$P3087S1_clean)
if (length(x) >= 2) {
  res_t1 <- t_test_media_desconocida(x, mu0 = mu0)
  print(res_t1)
  results$t_media <- res_t1
} else {
  cat("⚠️ Se requieren al menos 2 observaciones en P3087S1_clean.\n")
}


$t
[1] 23.53874

$p_value
[1] 0

$crit
[1] 1.967071

$df
[1] 335

$mean
[1] 998708.3

$s
[1] 621977.4

$se
[1] 33931.65

$ci
[1]  931962.4 1065454.3

$decision
[1] "Rechazar H0"



### Prueba 4: Diferencia de medias (σ conocidas) - Z

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una diferencia significativa entre las medias poblacionales de ahorro por cultivar (`P3094S3_clean`) y ahorro por criar animales (`P3095S3_clean`). Es adecuada porque comparamos dos variables cuantitativas independientes y asumimos conocer sus desviaciones estándar poblacionales.

**Hipótesis:**
- H₀: μ₁ - μ₂ = 0 (no hay diferencia entre el ahorro promedio por cultivar y por criar animales)
- H₁: μ₁ - μ₂ ≠ 0 (existe diferencia significativa entre ambos tipos de ahorro)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que el ahorro promedio por cultivar difiere significativamente del ahorro por criar animales. Esto sugiere que una actividad es más rentable que la otra.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que ambas actividades generan ahorros similares en promedio.

**Supuestos clave:**
- Ambas variables siguen distribución normal (o n₁, n₂ ≥ 30)
- Las desviaciones estándar poblacionales son conocidas (σ₁ = 300,000, σ₂ = 500,000)
- Las muestras son independientes
- Muestras aleatorias de sus respectivas poblaciones
- Las observaciones dentro de cada muestra son independientes

**Significado del estadístico y IC:**
- **Estadístico Z**: Mide cuántas desviaciones estándar se aleja la diferencia de medias muestrales de cero. Valores grandes indican evidencia contra H₀.
- **Intervalo de confianza**: Rango de valores plausibles para la diferencia de medias poblacionales. Si 0 no está en el IC, rechazamos H₀.


In [108]:
sigma1 <- 300000; sigma2 <- 500000
x1 <- na.omit(df$P3094S3_clean); x2 <- na.omit(df$P3095S3_clean)
if (length(x1) >= 1 && length(x2) >= 1) {
  res_z3 <- z_test_dif_medias_conocidas(x1, x2, sigma1 = sigma1, sigma2 = sigma2)
  print(res_z3)
  results$z_dif_medias <- res_z3
} else {
  cat("⚠️ No hay datos suficientes en las muestras para la prueba Z de diferencia de medias.\n")
}


$z
[1] -3.384629

$p_value
[1] 0.000712746

$crit
[1] 1.959964

$mean1
[1] 126937.6

$mean2
[1] 159471.4

$diff
[1] -32533.81

$se
[1] 9612.224

$ci
[1] -51373.42 -13694.20

$decision
[1] "Rechazar H0"



### Prueba 5: Diferencia de medias (varianzas iguales, desconocidas) - t (pooled)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una diferencia significativa entre las medias poblacionales del valor mensual por prácticas (`P3087S1_clean`) y el ahorro por cultivar (`P3094S3_clean`). Es adecuada porque comparamos dos variables cuantitativas independientes, asumimos varianzas poblacionales iguales pero desconocidas, y usamos la varianza combinada (pooled).

**Hipótesis:**
- H₀: μ₁ - μ₂ = 0 (no hay diferencia entre el valor promedio de prácticas y el ahorro promedio por cultivar)
- H₁: μ₁ - μ₂ ≠ 0 (existe diferencia significativa entre ambos valores)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que el valor promedio mensual por prácticas difiere significativamente del ahorro promedio por cultivar. Esto sugiere diferencias económicas importantes entre estas actividades.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que ambas actividades tienen valores económicos similares en promedio.

**Supuestos clave:**
- Ambas variables siguen distribución normal (o n₁, n₂ ≥ 30)
- Las varianzas poblacionales son iguales pero desconocidas (σ₁² = σ₂²)
- Las muestras son independientes
- Muestras aleatorias de sus respectivas poblaciones
- Se usa la varianza combinada (pooled) para estimar la varianza común

**Significado del estadístico y IC:**
- **Estadístico t**: Usa la varianza combinada y se distribuye como t con n₁ + n₂ - 2 grados de libertad.
- **Intervalo de confianza**: Rango de valores plausibles para la diferencia de medias usando la distribución t con gl combinados.


In [109]:
x1 <- na.omit(df$P3087S1_clean); x2 <- na.omit(df$P3094S3_clean)
if (length(x1) >= 2 && length(x2) >= 2) {
  res_t2 <- t_test_dif_medias_iguales(x1, x2)
  print(res_t2)
  results$t_pooled <- res_t2
} else {
  cat("⚠️ Se requieren al menos 2 observaciones en cada muestra para t pooled.\n")
}


$t
[1] 34.61998

$p_value
[1] 0

$crit
[1] 1.9607

$df
[1] 3226

$diff
[1] 871770.8

$sp2
[1] 190877666859

$se
[1] 25181.15

$ci
[1] 822398.1 921143.4

$decision
[1] "Rechazar H0"



### Prueba 6: Diferencia de medias (varianzas desiguales, desconocidas) - t de Welch

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una diferencia significativa entre las medias poblacionales del valor mensual por prácticas (`P3087S1_clean`) y el ahorro por criar animales (`P3095S3_clean`). Es adecuada porque comparamos dos variables cuantitativas independientes pero no asumimos que las varianzas poblacionales sean iguales, usando la aproximación de Welch.

**Hipótesis:**
- H₀: μ₁ - μ₂ = 0 (no hay diferencia entre el valor promedio de prácticas y el ahorro promedio por criar animales)
- H₁: μ₁ - μ₂ ≠ 0 (existe diferencia significativa entre ambos valores)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que el valor promedio mensual por prácticas difiere significativamente del ahorro promedio por criar animales. Esto sugiere diferencias económicas importantes entre estas actividades.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que ambas actividades tienen valores económicos similares en promedio.

**Supuestos clave:**
- Ambas variables siguen distribución normal (o n₁, n₂ ≥ 30)
- Las varianzas poblacionales pueden ser diferentes (σ₁² ≠ σ₂²)
- Las muestras son independientes
- Muestras aleatorias de sus respectivas poblaciones
- Se usa la aproximación de Welch para los grados de libertad

**Significado del estadístico y IC:**
- **Estadístico t**: Usa las varianzas muestrales por separado y se distribuye como t con grados de libertad de Welch (aproximados).
- **Intervalo de confianza**: Rango de valores plausibles para la diferencia de medias usando la distribución t con gl de Welch.


In [110]:
x1 <- na.omit(df$P3087S1_clean); x2 <- na.omit(df$P3095S3_clean)
if (length(x1) >= 2 && length(x2) >= 2) {
  res_t3 <- t_test_dif_medias_desiguales(x1, x2)
  print(res_t3)
  results$t_welch <- res_t3
} else {
  cat("⚠️ Se requieren al menos 2 observaciones en cada muestra para t de Welch.\n")
}


$t
[1] 24.06936

$p_value
[1] 0

$crit
[1] 1.966337

$df
[1] 373.4165

$diff
[1] 839237

$se
[1] 34867.44

$ci
[1] 770675.8 907798.1

$decision
[1] "Rechazar H0"



### Prueba 7: Diferencia de proporciones (Z)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una diferencia significativa en la proporción de asistencia a reuniones familiares (`P3101_bin`) entre dos grupos definidos por el valor mensual de prácticas (`P3087S1_clean`): grupo bajo (≤ mediana) vs grupo alto (> mediana). Es adecuada porque comparamos proporciones de una variable binaria entre dos grupos independientes.

**Hipótesis:**
- H₀: p₁ - p₂ = 0 (no hay diferencia en la proporción de asistencia a reuniones familiares entre grupos de bajo y alto valor de prácticas)
- H₁: p₁ - p₂ ≠ 0 (existe diferencia significativa en las proporciones de asistencia entre grupos)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que la proporción de asistencia a reuniones familiares difiere significativamente entre personas con bajo vs alto valor de prácticas. Esto sugiere una asociación entre el nivel económico (prácticas) y la participación familiar.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que la asistencia a reuniones familiares es similar en ambos grupos económicos.

**Supuestos clave:**
- La variable de respuesta es binaria (0/1)
- Los grupos son independientes
- Muestras aleatorias de cada grupo
- Tamaños muestrales suficientes: n₁p₁ ≥ 5, n₁(1-p₁) ≥ 5, n₂p₂ ≥ 5, n₂(1-p₂) ≥ 5
- Distribución muestral de la diferencia de proporciones aproximadamente normal

**Significado del estadístico y IC:**
- **Estadístico Z**: Mide cuántas desviaciones estándar se aleja la diferencia de proporciones muestrales de cero.
- **Intervalo de confianza**: Rango de valores plausibles para la diferencia de proporciones poblacionales.


In [111]:
comp <- df %>% select(P3101_bin, P3087S1_clean) %>% drop_na()
if (nrow(comp) >= 2) {
  med <- median(comp$P3087S1_clean)
  g1 <- comp %>% filter(P3087S1_clean <= med)
  g2 <- comp %>% filter(P3087S1_clean > med)
  x1 <- sum(g1$P3101_bin == 1L, na.rm = TRUE); n1 <- nrow(g1)
  x2 <- sum(g2$P3101_bin == 1L, na.rm = TRUE); n2 <- nrow(g2)
  if (n1 > 0 && n2 > 0) {
    res_z4 <- z_test_dif_proporciones(x1, n1, x2, n2)
    print(res_z4)
    results$z_dif_prop <- res_z4
  } else {
    cat("⚠️ Algún grupo quedó vacío tras filtrado.\n")
  }
} else {
  cat("⚠️ No hay datos suficientes para formar grupos.\n")
}


$z
[1] 0.4265208

$p_value
[1] 0.6697284

$crit
[1] 1.959964

$p1
[1] 0.01376147

$p2
[1] 0.008474576

$diff
[1] 0.005286892

$p_comb
[1] 0.01190476

$se
[1] 0.01239539

$ci
[1] -0.01735617  0.02792996

$decision
[1] "No rechazar H0"



### Prueba 8: Muestras dependientes (t pareado)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una diferencia significativa entre el ahorro por cultivar (`P3094S3_clean`) y el ahorro por criar animales (`P3095S3_clean`) en las mismas personas. Es adecuada porque tenemos observaciones pareadas (misma persona reporta ambos valores) y queremos evaluar si hay un cambio sistemático entre estas dos actividades.

**Hipótesis:**
- H₀: μd = 0 (no hay diferencia sistemática entre el ahorro por cultivar y por criar animales en la misma persona)
- H₁: μd ≠ 0 (existe diferencia sistemática entre ambos tipos de ahorro)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que las personas sistemáticamente ahorran más (o menos) en una actividad que en la otra. Esto sugiere que una actividad es consistentemente más rentable que la otra.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que no hay diferencia sistemática entre ambos tipos de ahorro en las mismas personas.

**Supuestos clave:**
- Las observaciones están pareadas (misma persona, mismo hogar)
- Las diferencias siguen distribución normal (o n ≥ 30)
- Las diferencias son independientes entre pares
- Muestra aleatoria de pares
- Se analiza la distribución de las diferencias, no las variables originales

**Significado del estadístico y IC:**
- **Estadístico t**: Se basa en la media y desviación estándar de las diferencias, con n-1 grados de libertad.
- **Intervalo de confianza**: Rango de valores plausibles para la media de las diferencias poblacionales.


In [112]:
pair_df <- df %>% select(P3094S3_clean, P3095S3_clean) %>% drop_na()
if (nrow(pair_df) >= 2) {
  res_t4 <- t_test_pareado(pair_df$P3094S3_clean, pair_df$P3095S3_clean)
  print(res_t4)
  results$t_pareado <- res_t4
} else {
  cat("⚠️ Se requieren al menos 2 pares válidos para t pareado.\n")
}


$t
[1] 0.6112736

$p_value
[1] 0.5411258

$crit
[1] 1.961798

$df
[1] 1295

$d_bar
[1] 9776.682

$s_d
[1] 575782.4

$se
[1] 15993.96

$ci
[1] -21600.22  41153.58

$decision
[1] "No rechazar H0"



### Prueba 9: Bondad de ajuste (Chi-cuadrado)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si la distribución observada de asistencia a reuniones familiares (`P3101_bin`) se ajusta a una distribución esperada uniforme (50% Sí, 50% No). Es adecuada porque tenemos una variable categórica y queremos verificar si sigue una distribución específica (uniforme en este caso).

**Hipótesis:**
- H₀: La distribución observada se ajusta a la distribución esperada (50% Sí, 50% No)
- H₁: La distribución observada no se ajusta a la distribución esperada

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de que la distribución real de asistencia a reuniones familiares difiere significativamente de la distribución uniforme esperada. Esto sugiere un patrón de asistencia no equilibrado en la población.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que la asistencia a reuniones familiares sigue una distribución uniforme (50%-50%).

**Supuestos clave:**
- La variable es categórica con k categorías
- Las observaciones son independientes
- Muestra aleatoria de la población
- Frecuencias esperadas ≥ 5 para todas las categorías
- Se compara la distribución observada con una distribución teórica específica

**Significado del estadístico y IC:**
- **Estadístico χ²**: Mide la discrepancia entre frecuencias observadas y esperadas. Valores grandes indican desajuste.
- **Grados de libertad**: k - 1, donde k es el número de categorías.


In [113]:
x <- df$P3101_bin
if (!is.null(x) && any(!is.na(x))) {
  res_chi2_1 <- chi2_bondad_ajuste(x, expected_props = c(0.5, 0.5))
  print(res_chi2_1)
  # Convertir tablas a listas para JSON
  res_chi2_1$observed <- as.list(res_chi2_1$observed)
  res_chi2_1$expected <- as.list(res_chi2_1$expected)
  results$chi2_gof <- res_chi2_1
} else {
  cat("⚠️ P3101_bin no disponible para bondad de ajuste.\n")
}


$chi2
[1] 154873.8

$p_value
[1] 0

$crit
[1] 3.841459

$df
[1] 1

$observed
x
     0      1 
163423   2918 

$expected
[1] 83170.5 83170.5

$decision
[1] "Rechazar H0"



### Prueba 10: Independencia (Chi-cuadrado)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe asociación entre la asistencia a reuniones familiares (`P3101_bin`) y el nivel de valor de prácticas (`P3087S1_clean` agrupado en terciles: Bajo, Medio, Alto). Es adecuada porque tenemos dos variables categóricas y queremos determinar si son independientes o están asociadas.

**Hipótesis:**
- H₀: Las variables son independientes (no hay asociación entre asistencia a reuniones familiares y nivel de valor de prácticas)
- H₁: Las variables están asociadas (existe dependencia entre asistencia a reuniones familiares y nivel de valor de prácticas)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de asociación entre la asistencia a reuniones familiares y el nivel económico (valor de prácticas). Esto sugiere que el nivel económico influye en la participación familiar.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de independencia. No hay evidencia de que el nivel económico afecte la asistencia a reuniones familiares.

**Supuestos clave:**
- Ambas variables son categóricas
- Las observaciones son independientes
- Muestra aleatoria de la población
- Frecuencias esperadas ≥ 5 en al menos el 80% de las celdas
- Se construye una tabla de contingencia (frecuencias cruzadas)

**Significado del estadístico y IC:**
- **Estadístico χ²**: Mide la discrepancia entre frecuencias observadas y esperadas bajo independencia. Valores grandes indican asociación.
- **Grados de libertad**: (r-1)(c-1), donde r y c son el número de filas y columnas.


In [114]:
ind <- df %>% select(P3101_bin, P3087S1_clean) %>% drop_na()
if (nrow(ind) >= 2) {
  q33 <- quantile(ind$P3087S1_clean, 0.33)
  q67 <- quantile(ind$P3087S1_clean, 0.67)
  grupo <- cut(ind$P3087S1_clean, breaks = c(-Inf, q33, q67, Inf), labels = c("Bajo", "Medio", "Alto"))
  asist <- ind$P3101_bin
  res_chi2_ind <- chi2_independencia(grupo, asist)
  print(res_chi2_ind)
  # Convertir tablas a listas para JSON
  res_chi2_ind$observed <- as.list(res_chi2_ind$observed)
  res_chi2_ind$expected <- as.list(res_chi2_ind$expected)
  results$chi2_ind <- res_chi2_ind
} else {
  cat("⚠️ No hay datos suficientes para la prueba de independencia.\n")
}


$chi2
[1] 0.2573086

$p_value
[1] 0.8792779

$df
[1] 2

$expected
       y
x              0        1
  Bajo  125.4881 1.511905
  Medio 100.7857 1.214286
  Alto  105.7262 1.273810

$observed
       y
x         0   1
  Bajo  125   2
  Medio 101   1
  Alto  106   1

$crit
[1] 5.991465

$decision
[1] "No rechazar H0"



### Prueba 11: Prueba de signos (muestras pareadas)

**¿Qué contrasta esta prueba?**
Esta prueba evalúa si existe una tendencia sistemática en las diferencias entre los rankings de ahorro por cultivar (`P3094S3_clean`) y ahorro por criar animales (`P3095S3_clean`) en las mismas personas. Es adecuada porque tenemos observaciones pareadas y queremos detectar sesgos direccionales sin asumir normalidad.

**Hipótesis:**
- H₀: No hay diferencia sistemática de signos (el número de diferencias positivas y negativas es similar)
- H₁: Existe diferencia sistemática de signos (hay más diferencias en una dirección que en la otra)

**Interpretación contextual:**
- **Rechazar H₀**: Hay evidencia de una tendencia sistemática en las diferencias entre rankings. Esto sugiere que una actividad tiende a tener rankings consistentemente más altos que la otra.
- **No rechazar H₀**: Los datos son consistentes con la hipótesis de que no hay tendencia sistemática. Las diferencias de ranking son aleatorias.

**Supuestos clave:**
- Las observaciones están pareadas
- Las diferencias son independientes entre pares
- Se ignoran las diferencias exactamente iguales (empates)
- No se requiere normalidad de las diferencias
- Se basa únicamente en los signos de las diferencias, no en sus magnitudes

**Significado del estadístico y IC:**
- **Estadístico**: Número mínimo de signos positivos o negativos. Se distribuye como binomial con p = 0.5.
- **p-valor**: Probabilidad de observar al menos tantas diferencias en la dirección menos frecuente bajo H₀.


In [115]:
pair_df <- df %>% select(P3094S3_clean, P3095S3_clean) %>% drop_na()
if (nrow(pair_df) >= 1) {
  r1 <- rank(pair_df$P3094S3_clean, ties.method = "average")
  r2 <- rank(pair_df$P3095S3_clean, ties.method = "average")
  res_signos <- prueba_signos(r1, r2)
  print(res_signos)
  results$signos <- res_signos
} else {
  cat("⚠️ No hay pares válidos para la prueba de signos.\n")
}


$stat
[1] 616

$p_value
[1] 0.08486947

$pos
[1] 679

$neg
[1] 616

$emp
[1] 1

$n
[1] 1295

$decision
[1] "No rechazar H0"



In [116]:
# Exportar resultados a JSON
json_path <- "resultados_pruebas.json"
jsonlite::write_json(results, json_path, pretty = TRUE, auto_unbox = TRUE)
cat("✅ Resultados exportados a:", json_path, "\n")

# Resumen de pruebas ejecutadas
cat("\n📊 RESUMEN DE PRUEBAS EJECUTADAS:\n")
cat("================================\n")
for (i in seq_along(results)) {
  test_name <- names(results)[i]
  decision <- results[[i]]$decision
  p_val <- results[[i]]$p_value
  cat(sprintf("%d. %s: %s (p=%.6f)\n", i, test_name, decision, p_val))
}


✅ Resultados exportados a: resultados_pruebas.json 

📊 RESUMEN DE PRUEBAS EJECUTADAS:
1. z_proporcion: Rechazar H0 (p=0.000000)
2. t_media: Rechazar H0 (p=0.000000)
3. z_dif_medias: Rechazar H0 (p=0.000713)
4. t_pooled: Rechazar H0 (p=0.000000)
5. t_welch: Rechazar H0 (p=0.000000)
6. z_dif_prop: No rechazar H0 (p=0.669728)
7. t_pareado: No rechazar H0 (p=0.541126)
8. chi2_gof: Rechazar H0 (p=0.000000)
9. chi2_ind: No rechazar H0 (p=0.879278)
10. signos: No rechazar H0 (p=0.084869)
